In [ ]:
#import labrary
from __future__ import print_function, division
from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob


In [ ]:
#Path dataset
covid_path = '/content/drive/MyDrive/Colab Notebooks/Xray dataset/COVID-19'
noncovid_path = '/content/drive/MyDrive/Colab Notebooks/Xray dataset/Normal'

# Mengambil gambar dari path 
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [ ]:
#Preparing labels 
covid_labels = []
noncovid_labels = []

covid_images=[]
noncovid_images=[]

import cv2 

#resize dan memberikan label pada dataset
for i in range(len(covid_files)):
  image = cv2.imread(covid_files[i])
  image = cv2.resize(image,(224,224))
  covid_images.append(image)
  covid_labels.append('Chest_COVID')
for i in range(len(noncovid_files)):
  image = cv2.imread(noncovid_files[i])
  image = cv2.resize(image,(224,224))
  noncovid_images.append(image)
  noncovid_labels.append('Chest_NonCOVID')

In [ ]:
# Menampilkan gambar dataset covid dan noncovid secara acak
def plot_images(images, title):
    nrows, ncols = 5, 8
    figsize = [10, 6]

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, facecolor=(1, 1, 1))

    for i, axi in enumerate(ax.flat):
        axi.imshow(images[i])
        axi.set_axis_off()

    plt.suptitle(title, fontsize=24)
    plt.tight_layout(pad=0.2, rect=[0, 0, 1, 0.9])
    plt.show()
plot_images(covid_images, 'Positive COVID-19 Chest X-ray')
plot_images(noncovid_images, 'Negative COVID-19 Chest X-ray')

In [ ]:
# normalisasi [0,1]
#covid_images = np.array(covid_images) / 255
#noncovid_images = np.array(noncovid_images) / 255


#normalisasi [-1,1]
covid_images_norm=[]
noncovid_images_norm=[]
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

for i in range(len(covid_images)):
  imagenorm = []
  for j in np.array(covid_images[i]):
    partimage = (np.array(scaler.fit_transform(j)))
    imagenorm.append(partimage)
  covid_images_norm.append(np.array(imagenorm))

for i in range(len(noncovid_images)):
  imagenorm = []
  for j in np.array(noncovid_images[i]):
    partimage = (np.array(scaler.fit_transform(j)))
    imagenorm.append(partimage)
  noncovid_images_norm.append(imagenorm)

In [ ]:
# Augmentasi dataset
train_datagen = ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
    )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

# split into training and testing
covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(
    covid_images_norm, covid_labels, test_size=0.3)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(
    noncovid_images_norm, noncovid_labels, test_size=0.3)

covid_x_train, covid_x_val, covid_y_train, covid_y_val = train_test_split(
    covid_x_train, covid_y_train, test_size=0.2)
noncovid_x_train, noncovid_x_val, noncovid_y_train, noncovid_y_val = train_test_split(
    noncovid_x_train, noncovid_y_train, test_size=0.2)

X_train = np.concatenate((noncovid_x_train, covid_x_train), axis=0)
X_test = np.concatenate((noncovid_x_test, covid_x_test), axis=0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis=0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis=0)
X_val = np.concatenate((noncovid_x_val, covid_x_val), axis=0)
y_val = np.concatenate((noncovid_y_val, covid_y_val), axis=0)

#Membuat label menjadi 0 atau 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

y_val = LabelBinarizer().fit_transform(y_val)
y_val = to_categorical(y_val)